# 2.1 Adding Indexes with the DMVs

### The 'A' part relies on Clippy

What are clippy's sources?
- sys.dm_db_missing_index_details
- Query plans and the plan cache

He doesn't care about
- The size of the index
- The index's overhead on D / U / I operations
- Your other existing indexes
- A lot of operations in the plan (GROUP BY)

Clippys ordering
1. Equality searches in the query (ordered by field order of the table)
2. Inequality searches (ordered by field order in table)
3. Includes (might need to be sorted)

**Activity #1** Produce a query which returns a missing index recommendation which uses (DisplayName, Location)



In [ ]:
-- My answer, using a pair of equality searches
SELECT DisplayName, Location
FROM dbo.Users
WHERE Location LIKE 'Adam%'
AND DisplayName IN ('Seattle%')

-- Another answer, using an equality and inequality search
SELECT * 
FROM dbo.Users 
WHERE DisplayName = 'asdf' 
AND Location <> 'New Zealand'

-- Brent's queries

SELECT Id
FROM dbo.Users
WHERE DisplayName > 'Brent Ozar'
AND Location IS NOT NULL;

SELECT Id
FROM dbo.Users
WHERE DisplayName = 'Alex'
AND Location = 'India'
ORDER BY Reputation

- Note: Clippy generally doesnt care about the GROUP BY or ORDER BY... except when it does (ie. it depends...)

**Activity #2:** Produce a query which generates an EQUALITY search on Location, and an INEQUALITY search on DisplayName, and INCLUDES all the columns

In [ ]:
/* My / Brents query */ 
SELECT TOP 100 *
FROM dbo.Users
WHERE Location = 'Seattle, WA'
AND DisplayName <> '%Brent Ozar%'


**Activity #3** - Produce a query that generates a recommendation that contains DisplayName, but just INCLUDE's location

In [ ]:
SELECT TOP 100 Location
FROM dbo.Users
WHERE DisplayName <> '%Brent Ozar%'
ORDER BY Location

**Activity #4** Find an index that covers BOTH of the following missing index requests:

In [ ]:
-- CREATE NONCLUSTED INDEX [Name] ON [dbo].[Users] (LastAccessDate, WebsiteUrl)

-- CREATE NONCLUSTED INDEX [Name] ON [dbo].[Users] (DownVotes, WebsiteUrl)

Note that the missing index recommendations include the following XML:

<img src="C:\Users\hartleyg\Desktop\Training\sqltraining\Brent Ozar\Mastering Index Tuning\2.1 Adding Indexes with the DMVs\2.1.01.png" width=400>

**Ask:**
1. What queries might have built these hints?
2. Can one index satisfy both queries? Why?

Since the XML shows that these are both EQUALITY searches, recall that that ordering doesnt matter. Therefore the following index will cover both queries:

In [ ]:
CREATE INDEX WebsiteUrl_LastAccessDate_DownVotes
    ON dbo.Users(WebsiteUrl, LastAccessDate, DownVotes);

While this Index improves performance on both queries, it doesn't provide the best possible performance for each individual query.

If we were to change the key ordering to suit the key order of one particular query, we would expect poor performance with the other query. 

However, it will opt to use this index as this provides the smallest copy of the table that contains the fields that it requires. <br>

## Our job with index tuning

Look at the recommended indexes by:
- Equality fields
- Inequality fields
- Includes (which sometimes need to be promoted)

Look for things they have in common

Make guess based on our exprience with the queries and the data

> By the time you reach the 'T' part of optimisation, you should start to know the data a lot better

Revisit it later in the T part of D.E.A.T.H

## Field ordering guidelines (not rules)

Fields you use the most often should go first

When doing range scans, selectivity matters

Comprimise involves:
- Prioritizing reads vs writes
- Prioritizing which queries need to be faster
- Caching data in the application
- Spending more money on hardware

